В данном ноутбуке представлены только модели участника команды, который получил наилучший результат на имеющихся данных

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import norm, uniform, beta, lognorm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_columns', None)
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.metrics import roc_auc_score
#import optuna
from tqdm import tqdm
from IPython.display import clear_output
from scipy.stats import norm
import warnings
warnings.simplefilter("ignore")

In [2]:
df_main = pd.read_excel('rating_dataset.xlsx')

In [3]:
df_main.sort_values(['company', 'date'], inplace=True)
df_main['default'] = df_main['default'].replace({'отозв.': 1, 'ликв.': 1}).astype(float)

# Сброс индекса
df_main.reset_index(drop=True, inplace=True)

mask = (df_main['default'].diff() == -1)

same_company = df_main['company'] == df_main['company'].shift()

result = df_main[mask & same_company]
df_main = df_main.drop(result.index)

df_main.sort_values(['company', 'date'], inplace=True)
df_main.reset_index(drop=True, inplace=True)

In [4]:
df_main_finance = df_main[['date', 'company', 'number', 'net_assets', 'ROA', 'liquid', 'ibl',
       'stocks', 'bond', 'oth_cap', 'sunk_retail_credit', 'NI',
       'organization_credit', 'sunk_organization_credit', 'credit_portf',
       'sunk_credit_portf', 'organization_deposit', 'retail_deposit',
       'security_tot', 'ROE', 'retail_credit', 'reserv_credit_perc',
       'zalog_credit_perc', 'foreign_na_fr', 'retail_deposit_fr', 'N3', 'N2',
       'N1', 'capital', 'msk_spb', 'INF_SA', 'NX_growth', 'micex_std',
       'miacr_std', 'miacr_amount', 'usd_rub_std_diff', 'micex_return',
       'net_foreign_assets_diff', 'net_gov_debt_diff', 'other_fin_debt_diff',
       'retail_debt_SA_DETREND_diff', 'stocks_capital_diff',
       'i_retail_spread_diff', 'usd_rub_return', 'miacr_diff', 'default']]

In [5]:
oot = df_main_finance[df_main_finance['date'] >= '2017-05']
train = df_main_finance[df_main_finance['date'] < '2017-05']


In [6]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(['date', 'company', 'number', 'default'], axis=1),
                                                  train.default,
                                                  test_size=0.2,
                                                  random_state=42)

In [7]:
logreg = LogisticRegression(max_iter=1000)

logreg.fit(X_train, y_train)

y_val_pred = logreg.predict_proba(X_val)[:, 1]


roc_auc_val = roc_auc_score(y_val, y_val_pred)
print(f'ROC-AUC валид: {roc_auc_val}')


y_oot_pred = logreg.predict_proba(oot.drop(['date', 'company', 'number', 'default'], axis=1))[:, 1]
roc_auc_oot = roc_auc_score(oot.default, y_oot_pred)
print(f'ROC-AUC тест: {roc_auc_oot}')

ROC-AUC валид: 0.5666655440676255
ROC-AUC тест: 0.5769476372924648


In [ ]:
oot = df_main_finance[df_main_finance['date'] >= '2017-05']
train = df_main_finance[df_main_finance['date'] < '2017-05']

X_train, X_val, y_train, y_val = train_test_split(train.drop(['date', 'company', 'number', 'default'], axis=1),
                                                  train.default,
                                                  test_size=0.2,
                                                  random_state=42)


xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    eval_metric='auc',
    use_label_encoder=False,
    random_state=42,
    verbose=False
)
xgb_model.fit(X_train, y_train)

thresholds = sorted(xgb_model.feature_importances_, reverse=True)[25]
sfm = SelectFromModel(xgb_model, threshold=thresholds)
X_train_sfm = sfm.fit_transform(X_train, y_train)
X_val_sfm = sfm.transform(X_val)


xgb_model.fit(X_train_sfm, y_train, eval_set=[(X_val_sfm, y_val)], early_stopping_rounds=10, verbose=True)


y_train_pred = xgb_model.predict_proba(X_train_sfm)[:, 1]

# Вычисление ROC-AUC для обучающего набора данных


y_val_pred = xgb_model.predict_proba(X_val_sfm)[:, 1]




X_oot = oot.drop(['date', 'company', 'number', 'default'], axis=1)
X_oot_sfm = sfm.transform(X_oot)
y_oot_pred = xgb_model.predict_proba(X_oot_sfm)[:, 1]

In [ ]:
roc_auc_train = roc_auc_score(y_train, y_train_pred)
print(f'ROC-AUC обуч: {roc_auc_train}')
roc_auc_val = roc_auc_score(y_val, y_val_pred)
print(f'ROC-AUC валид: {roc_auc_val}')
roc_auc_oot = roc_auc_score(oot.default, y_oot_pred)
print(f'ROC-AUC тест: {roc_auc_oot}')

In [ ]:
filename = 'model_fin.pkl'
pickle.dump(xgb_model, open(filename, 'wb'))

filename = 'sfm.pkl'
pickle.dump(sfm, open(filename, 'wb'))

In [ ]:
feature_importances = xgb_model.feature_importances_

column_names = X_train.columns[sfm.get_support()]

importance_data = sorted(zip(column_names, feature_importances), key=lambda x: x[1], reverse=True)

for col_name, importance in importance_data[:30]:
    print(f'Feature: {col_name}, Importance: {importance}')

In [12]:
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'mlogloss',
        'use_label_encoder': False,
        'random_state': 42
    }

    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train, y_train)

    y_pred = optuna_model.predict_proba(X_val)

    roc_auc = roc_auc_score(y_val, y_pred[:, 1])
    return roc_auc

In [13]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2024-03-20 23:46:07,434] A new study created in memory with name: no-name-0ac817eb-da70-4c79-997c-f158e4d1aa4c
[I 2024-03-20 23:46:08,111] Trial 0 finished with value: 0.5748829690499444 and parameters: {'max_depth': 1, 'learning_rate': 0.022876653476122614, 'n_estimators': 209, 'min_child_weight': 1, 'gamma': 8.56004362165115e-06, 'subsample': 0.35348853734636254, 'colsample_bytree': 0.06885846484783337, 'reg_alpha': 0.002206102762049319, 'reg_lambda': 0.015682571761648896}. Best is trial 0 with value: 0.5748829690499444.
[I 2024-03-20 23:46:08,321] Trial 1 finished with value: 0.5431583201427946 and parameters: {'max_depth': 6, 'learning_rate': 0.018076841499726517, 'n_estimators': 66, 'min_child_weight': 5, 'gamma': 0.028760196845221325, 'subsample': 0.023595033868409086, 'colsample_bytree': 0.18171712048299396, 'reg_alpha': 3.9430209426487994e-06, 'reg_lambda': 0.22091566271703836}. Best is trial 0 with value: 0.5748829690499444.
[I 2024-03-20 23:46:09,040] Trial 2 finished with

In [14]:
print('Best hyperparameters:', study.best_params)
print('Best GINI:', study.best_value)

Best hyperparameters: {'max_depth': 9, 'learning_rate': 0.05967988601394193, 'n_estimators': 303, 'min_child_weight': 1, 'gamma': 0.0016228946911716325, 'subsample': 0.9470352098444895, 'colsample_bytree': 0.0115855131025056, 'reg_alpha': 1.4530122502856077e-08, 'reg_lambda': 0.0004090059013383458}
Best GINI: 0.8091806149597548


In [ ]:
xgb_model2 = XGBClassifier(**study.best_params)
xgb_model2.fit(X_train, y_train)

thresholds2 = sorted(xgb_model2.feature_importances_, reverse=True)[25]
sfm2 = SelectFromModel(xgb_model2, threshold=thresholds)
X_train_sfm2 = sfm2.fit_transform(X_train, y_train)
X_val_sfm2 = sfm2.transform(X_val)


xgb_model2.fit(X_train_sfm2, y_train, eval_set=[(X_val_sfm2, y_val)], early_stopping_rounds=10, verbose=True)


y_train_pred = xgb_model2.predict_proba(X_train_sfm2)[:, 1]
y_val_pred = xgb_model2.predict_proba(X_val_sfm2)[:, 1]

X_oot = oot.drop(['date', 'company', 'number', 'default'], axis=1)
X_oot_sfm2 = sfm2.transform(X_oot)
y_oot_pred = xgb_model2.predict_proba(X_oot_sfm2)[:, 1]

In [ ]:
roc_auc_train = roc_auc_score(y_train, y_train_pred)
print(f'ROC-AUC обуч: {roc_auc_train}')
roc_auc_val = roc_auc_score(y_val, y_val_pred)
print(f'ROC-AUC валид: {roc_auc_val}')
roc_auc_oot = roc_auc_score(oot.default, y_oot_pred)
print(f'ROC-AUC тест: {roc_auc_oot}')

Оптимизация гиперпараметров с помощью Optuna не улучшила результат

In [ ]:
df_feature_importances = pd.DataFrame({"feature": column_names, "importance": feature_importances}).sort_values(by="importance", ascending=False)

# Построить график
plt.figure(figsize=(15, 8))
sns.barplot(data=df_feature_importances[:30], x="importance", y="feature")
plt.title("Feature importance, GBC", fontsize=20)
plt.show()

In [ ]:
explainer = shap.Explainer(xgb_model)
shap_values = explainer(X_train_sfm)

shap.plots.beeswarm(shap_values=shap_values, plot_size=(15, 8))

shap.plots.waterfall(shap_values[2])

In [ ]:
calibration = df_main_finance[(df_main_finance['date'] < '2017-05') & (df_main_finance['date'] >= '2017-02')]

X_calibration = calibration.drop(['date', 'company', 'number', 'default'], axis=1)
X_calibration_sfm = sfm.transform(X_calibration)


y_calibration = calibration['default']

proba_calibration = xgb_model.predict_proba(X_calibration_sfm)[:, 1]

isotonic = IsotonicRegression(y_min=0, y_max=1, out_of_bounds='clip')
isotonic.fit(proba_calibration, y_calibration)

proba_isotonic = isotonic.predict(xgb_model.predict_proba(X_oot_sfm)[:, 1])

logistic = LogisticRegression()
logistic.fit(proba_calibration.reshape(-1, 1), y_calibration)

proba_logistic = logistic.predict_proba(xgb_model.predict_proba(X_oot_sfm)[:, 1].reshape(-1, 1))[:, 1]


def expected_calibration_error(y, proba, bins = 'fd'):
  bin_count, bin_edges = np.histogram(proba, bins = bins)
  n_bins = len(bin_count)
  bin_edges[0] -= 1e-8 # because left edge is not included
  bin_id = np.digitize(proba, bin_edges, right = True) - 1
  bin_ysum = np.bincount(bin_id, weights = y, minlength = n_bins)
  bin_probasum = np.bincount(bin_id, weights = proba, minlength = n_bins)
  bin_ymean = np.divide(bin_ysum, bin_count, out = np.zeros(n_bins), where = bin_count > 0)
  bin_probamean = np.divide(bin_probasum, bin_count, out = np.zeros(n_bins), where = bin_count > 0)
  ece = np.abs((bin_probamean - bin_ymean) * bin_count).sum() / len(proba)
  return ece

exp_cal_er_xg = expected_calibration_error(oot.default, xgb_model.predict_proba(X_oot_sfm)[:, 1])
print('ECE откалиброванной модели XGBoost на ООТ = ' + str(round(exp_cal_er_xg, 4)))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(['date', 'company', 'number', 'default'], axis=1),
                                                  train.default,
                                                  test_size=0.2,
                                                  random_state=42)


xgb_model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    eval_metric='auc',
    use_label_encoder=False,
    random_state=42,
    verbose=False
)
xgb_model.fit(X_train, y_train)

thresholds = sorted(xgb_model.feature_importances_, reverse=True)[25]
sfm = SelectFromModel(xgb_model, threshold=thresholds)
X_train_sfm = sfm.fit_transform(X_train, y_train)
X_val_sfm = sfm.transform(X_val)


xgb_model.fit(X_train_sfm, y_train, eval_set=[(X_val_sfm, y_val)], early_stopping_rounds=10, verbose=True)


y_train_pred = xgb_model.predict_proba(X_train_sfm)[:, 1]

# Вычисление ROC-AUC для обучающего набора данных


y_val_pred = xgb_model.predict_proba(X_val_sfm)[:, 1]




X_oot = oot.drop(['date', 'company', 'number', 'default'], axis=1)
X_oot_sfm = sfm.transform(X_oot)
oot['pd'] = xgb_model.predict_proba(X_oot_sfm)[:, 1]

## Расчет для 6 задания

In [21]:
avangard_bank = oot[oot['number'] == 2879]
expo_bank = oot[oot['number'] == 2998]
ros_bank = oot[oot['number'] == 2272]
alpha_bank = oot[oot['number'] == 1326]
loko_bank = oot[oot['number'] == 2707]

### Горизонт 1 год

In [22]:
n_simulations = 1000000

lgd=0.4

PD = [1 - (1 - avangard_bank.iloc[-1]['pd']**4),
      1 - (1 - expo_bank.iloc[-1]['pd'])**4,
      1 - (1 - ros_bank.iloc[-1]['pd'])**4,
      1 - (1 - alpha_bank.iloc[-1]['pd'])**4,
      1 - (1 - loko_bank.iloc[-1]['pd'])**4]

avangard_loss = np.random.binomial(n=1, p=PD[0], size=[n_simulations]) * 200000000 * lgd
expobank_loss = np.random.binomial(n=1, p=PD[1], size=[n_simulations]) * 200000000 * lgd
rosbank_loss = np.random.binomial(n=1, p=PD[2], size=[n_simulations]) * 200000000 * lgd
alfabank_loss = np.random.binomial(n=1, p=PD[3], size=[n_simulations]) * 200000000 * lgd
lokobank_loss = np.random.binomial(n=1, p=PD[4], size=[n_simulations]) * 200000000 * lgd

total_loss = map(sum, zip(avangard_loss, expobank_loss, rosbank_loss, alfabank_loss, lokobank_loss))

loss_df = pd.DataFrame({'Loss': list(total_loss)}).sort_values(by='Loss', ascending=False)
loss_df.describe(percentiles=[0.999]).T

,count,mean,std,min,50%,99.9%,max
Loss,1000000.0,36315600.0,5.050702e+07,0.0,0.0,240000000.0,320000000.0


### Горизонт 5 лет

In [18]:
n_simulations = 1000000

lgd=0.4

PD5 = [1 - (1 - avangard_bank.iloc[-1]['pd']**20),
      1 - (1 - expo_bank.iloc[-1]['pd'])**20,
      1 - (1 - ros_bank.iloc[-1]['pd'])**20,
      1 - (1 - alpha_bank.iloc[-1]['pd'])**20,
      1 - (1 - loko_bank.iloc[-1]['pd'])**20]

avangard_loss5 = np.random.binomial(n=1, p=PD5[0], size=[n_simulations]) * 200000000 * lgd
expobank_loss5 = np.random.binomial(n=1, p=PD5[1], size=[n_simulations]) * 200000000 * lgd
rosbank_loss5 = np.random.binomial(n=1, p=PD5[2], size=[n_simulations]) * 200000000 * lgd
alfabank_loss5 = np.random.binomial(n=1, p=PD5[3], size=[n_simulations]) * 200000000 * lgd
lokobank_loss5 = np.random.binomial(n=1, p=PD5[4], size=[n_simulations]) * 200000000 * lgd

total_loss5 = map(sum, zip(avangard_loss5, expobank_loss5, rosbank_loss5, alfabank_loss5, lokobank_loss5))
loss_df = pd.DataFrame({'Loss': list(total_loss5)}).sort_values(by='Loss', ascending=False)
loss_df.describe(percentiles=[0.999]).T

NameError: name 'avangard_bank' is not defined

### Задание b вариант 1

In [16]:
def get_maturity_slope(pd, a=0.11852, b=-0.05478):
    """
        DESCRIPTION:
        ------------
        Credit portfolios consist of instruments with different maturities. Both intuition and
        empirical evidence indicate that long-term credits are riskier than short-term credits. As a
        consequence, the capital requirement should increase with maturity.

        In order to derive the Basel maturity adjustment function, the grid of relative VaR figures
        (in relation to 2.5 years maturity) was smoothed by a statistical regression model.
        This includes the slope of the adjustment function with respect to M decreases as
        the PD increases.

        REFERENCE:
        ----------
        https://www.bis.org/bcbs/irbriskweight.pdf
        4.6. Maturity adjustments

        PARAMS:
        -------
        :param pd: array of PDs from portfolio
        :param a: slope adjustment coefficient 1
        :param b: slope adjustement coefficient 2
        :return maturity_slope: array of Smoothed (regression) maturity adjustment (smoothed over PDs)
    """
    return np.power((a + b * np.log(pd)), 2)


def get_maturity_adjusment(pd, m, y=2.5):
    """
        DESCRIPTION:
        ------------
        Maturity adjustments are the ratios of each of these VaR figures to the VaR
        of a “standard” maturity, which was set at 2.5 years, for each maturity and each rating grade.

        REFERENCE:
        ----------
        https://www.bis.org/bcbs/irbriskweight.pdf
        4.6. Maturity adjustments

        PARAMS:
        -------
        :param pd: array of PDs from portfolio
        :param tenor: array of tenors from portfolio or remaining term to maturity
        :param y: standard maturity, 2.5 years
        :return maturity_adj: array of maturity adjustments used in risk-capital allocation
    """
    slope = get_maturity_slope(pd)
    return np.divide(1 + (m - y) * slope, 1 - 1.5 * slope)

In [17]:
pd_array = np.array(PD5)

m = 5 #maturity adjustment

maturity_adj = get_maturity_adjusment(pd_array, m, y=2.5)
avangard_loss5 = np.random.binomial(n=1, p=PD5[0], size=[n_simulations]) * 200000000 * lgd * maturity_adj[0]
expobank_loss5 = np.random.binomial(n=1, p=PD5[1], size=[n_simulations]) * 200000000 * lgd * maturity_adj[1]
rosbank_loss5 = np.random.binomial(n=1, p=PD5[2], size=[n_simulations]) * 200000000 * lgd * maturity_adj[2]
alfabank_loss5 = np.random.binomial(n=1, p=PD5[3], size=[n_simulations]) * 200000000 * lgd * maturity_adj[3]
lokobank_loss5 = np.random.binomial(n=1, p=PD5[4], size=[n_simulations]) * 200000000 * lgd * maturity_adj[4]

total_loss5 = map(sum, zip(avangard_loss5, expobank_loss5, rosbank_loss5, alfabank_loss5, lokobank_loss5))
loss_df = pd.DataFrame({'Loss': list(total_loss5)}).sort_values(by='Loss', ascending=False)
loss_df.describe(percentiles=[0.999]).T

NameError: name 'n_simulations' is not defined

### Задание b вариант 2

In [4]:
def get_rho_asset_correlation(pd, k_factor=-50, rho_min=0.12, rho_max=0.24):
    """
        DESCRIPTION:
        -----------
        The single systematic risk factor needed in the ASRF model may be interpreted as reflecting
        the state of the global economy. The degree of the obligor’s exposure to the systematic risk
        factor is expressed by the asset correlation. The asset correlations, in short, show how the
        asset value (e.g. sum of all asset values of a firm) of one borrower depends on the asset
        value of another borrower.

        The asset correlation function is built of two limit correlations of 12% and 24% for very
        high and very low PDs (100% and 0%, respectively). Correlations between these limits are
        modelled by an exponential weighting function that displays the dependency on PD. The
        exponential function decreases rather fast; its pace is determined by the so-called
        “k-factor”, which is set at 50 for corporate exposures.

        REFERENCE:
        ----------
        https://www.bis.org/bcbs/irbriskweight.pdf
        5.2. Supervisory estimates of asset correlations for corporate, bank and sovereign exposures

        PARAMS:
        -------
        :param pd: arrays of PDs from portfolio
        :param k_factor: Set to 50 for corporates exposures
        :param rho_min: 12% min limit, for PDs = 100%
        :param rho_max: 14% max limit, for PDs = 0%
        :return rho: array of assets correlation
    """
    exponential_weights = np.divide((1 - np.exp(k_factor * pd)), (1 - np.exp(k_factor)))
    return rho_min * exponential_weights + rho_max * (1 - exponential_weights)

def conditional_pd(pd, rho, y):
    """
        DESCRIPTION:
        ------------
        The Vasicek model is a one period default model, i.e., loss only occurs when an 
        obligor defaults in a fixed time horizon. Based on Merton‘s firm-value model, to describe 
        the obligor’s default and its correlation structure, we assign each obligor a random
        variable called firm-value.

        The firm-value of obligor n is represented by a common, standard normally distributed factor
        Y component and an idiosyncratic standard normal noise component n.
        The Y factor is the state of the world or business cycle, usually called systematic factor.

        The probability of default of obligor n conditional to a realization of Y = y is given by
        this function.

        REFERENCE:
        ----------
        https://core.ac.uk/download/pdf/41778167.pdf

        PARAMS:
        -------
        :param pd: probability of default of obligor n
        :param rho: asset correlation, to be computed from IRB approach
        :param y: The Y factor is the state of the world or business cycle, usually called
        systematic factor.
        :return pd_conditional: conditional probability of default of obligor n to a realization of
        Y.
    """

    numerator = np.subtract(norm.ppf(pd), np.multiply(np.sqrt(rho), y))
    denominator = np.sqrt(1 - rho)
    return norm.cdf(np.divide(numerator, denominator))


def run_simulation(pd, ead, lgd, m, num_simulations=1000):
    """
        DESCRIPTION:
        ------------
        This functions runs a montecarlo simulation

        1) y is randomly generated from a normal distribution
        2) we generate a uniform continuous random variable
        3) we compare the random variable to pd to define if default there is
        4) if default compute loss = default_indicator * EAD * LGD * ma
        5) aggregate all losses
        PARAMS:
        -------
        :param pd: probability of default
        :param ead: exposure at risk
        :param lgd: loss given default
        :param num_simulations: number of simulation/iteration to be performed
        :return scenario_losses: array of losses
    """
    scenario_losses = np.array([])
    rho = get_rho_asset_correlation(pd)
    maturity_adj = get_maturity_adjusment(pd, m, y=2.5)
    for _ in range(num_simulations):
        y = np.random.normal(size=1)
        pd_cond = conditional_pd(pd, rho, y)
        uniform_random_variable = uniform.rvs(size=pd.size)
        default_indicator = (uniform_random_variable < pd_cond) * 1.0
        loss = default_indicator * ead * lgd * maturity_adj
        total_loss = loss.sum()
        scenario_losses = np.concatenate([scenario_losses, np.array([total_loss])])
    return scenario_losses

In [5]:
PD5 = [0.1, 0.52982909, 0.53439636, 0.34748718, 0.37568186]
rho = get_rho_asset_correlation(PD5)
rho

array([], dtype=float64)

In [38]:
ead=200000000
scenario_losses = run_simulation(pd_array, ead, lgd, m, 100)
scenario_losses = np.mean(scenario_losses)
var_999 = np.percentile(scenario_losses, 0.1)

In [39]:
scenario_losses

145377806.66063532

### Задание 3

In [9]:
rho = get_rho_asset_correlation(pd_array)
WCDR = norm.cdf(np.sqrt(1/(1-rho)) * norm.ppf(PD) + np.sqrt(rho/(1-rho)) * norm.ppf(0.999))
K = (WCDR - PD) * lgd * ead 
K

ValueError: operands could not be broadcast together with shapes (0,) (5,) 

In [41]:
K.sum()

103490071.17581475

In [6]:
pd_array = [1.3963097936775526e-06,
 0.14009366093479736,
 0.1417708275062729,
 0.08184118064241586,
 0.08991660534671952]
lgd = 0.4
ead = 200000000

In [13]:
pd_array

[1.3963097936775526e-06,
 0.14009366093479736,
 0.1417708275062729,
 0.08184118064241586,
 0.08991660534671952]

In [14]:
rho = [0.12, 0.200238562, 0.202498572, 0.163330245, 0.17687558]

In [11]:
pd_random = [0.5, 0.31, 0.2, 0.5, 0.3]

In [12]:
rho = get_rho_asset_correlation(pd_random)
rho

array([], dtype=float64)